In [1]:
%load_ext autoreload


In [2]:
%autoreload 2


In [3]:
import os
import sys
import importlib

sys.path.append(os.path.abspath('../scripts'))

from Feature_Engineering import Feature_Engineering
from Eda import CreditScoringEDA





In [4]:

path = '../data/data.csv'
fe = Feature_Engineering(path)

fe.data.head()


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [6]:
fe.total_transaction_amount()


Filtered Data for Total Transaction Amount:
           TransactionId         BatchId       AccountId       SubscriptionId  \
0   TransactionId_76871   BatchId_36123  AccountId_3957   SubscriptionId_887   
2   TransactionId_26203   BatchId_53941  AccountId_4229   SubscriptionId_222   
3     TransactionId_380  BatchId_102363   AccountId_648  SubscriptionId_2185   
5   TransactionId_23223   BatchId_25954  AccountId_1078  SubscriptionId_4238   
6  TransactionId_118063  BatchId_118460  AccountId_2442  SubscriptionId_1980   

        CustomerId CurrencyCode  CountryCode    ProviderId     ProductId  \
0  CustomerId_4406          UGX          256  ProviderId_6  ProductId_10   
2  CustomerId_4683          UGX          256  ProviderId_6   ProductId_1   
3   CustomerId_988          UGX          256  ProviderId_1  ProductId_21   
5  CustomerId_1432          UGX          256  ProviderId_6   ProductId_3   
6  CustomerId_2858          UGX          256  ProviderId_5   ProductId_3   

  ProductCategory

,CustomerId,total_transaction_amount
0,CustomerId_1001,25200.0
1,CustomerId_1002,4500.0
2,CustomerId_1003,26000.0
3,CustomerId_1004,2000.0
4,CustomerId_1005,55000.0
...,...,...
3627,CustomerId_992,26000.0
3628,CustomerId_993,26000.0
3629,CustomerId_994,578000.0
3630,CustomerId_996,145000.0


aggregates the transaction amounts for each customer. Specifically, it groups the data by CustomerId and calculates the sum of the Amount for each customer

In [6]:
fe.average_transaction_amount()

Filtered Data for Average Transaction Amount:
           TransactionId         BatchId       AccountId       SubscriptionId  \
0   TransactionId_76871   BatchId_36123  AccountId_3957   SubscriptionId_887   
2   TransactionId_26203   BatchId_53941  AccountId_4229   SubscriptionId_222   
3     TransactionId_380  BatchId_102363   AccountId_648  SubscriptionId_2185   
5   TransactionId_23223   BatchId_25954  AccountId_1078  SubscriptionId_4238   
6  TransactionId_118063  BatchId_118460  AccountId_2442  SubscriptionId_1980   

        CustomerId CurrencyCode  CountryCode    ProviderId     ProductId  \
0  CustomerId_4406          UGX          256  ProviderId_6  ProductId_10   
2  CustomerId_4683          UGX          256  ProviderId_6   ProductId_1   
3   CustomerId_988          UGX          256  ProviderId_1  ProductId_21   
5  CustomerId_1432          UGX          256  ProviderId_6   ProductId_3   
6  CustomerId_2858          UGX          256  ProviderId_5   ProductId_3   

  ProductCatego

,CustomerId,average_transaction_amount
0,CustomerId_1001,8400.000000
1,CustomerId_1002,900.000000
2,CustomerId_1003,6500.000000
3,CustomerId_1004,2000.000000
4,CustomerId_1005,9166.666667
...,...,...
3627,CustomerId_992,6500.000000
3628,CustomerId_993,8666.666667
3629,CustomerId_994,9475.409836
3630,CustomerId_996,9666.666667


In [5]:
from Feature_Engineering import  creating_aggregate_features

In [6]:
data_with_aggregates = creating_aggregate_features(fe.data)
print(data_with_aggregates)

              TransactionId         BatchId       AccountId  \
0       TransactionId_76871   BatchId_36123  AccountId_3957   
1       TransactionId_73770   BatchId_15642  AccountId_4841   
2       TransactionId_26203   BatchId_53941  AccountId_4229   
3         TransactionId_380  BatchId_102363   AccountId_648   
4       TransactionId_28195   BatchId_38780  AccountId_4841   
...                     ...             ...             ...   
95657   TransactionId_89881   BatchId_96668  AccountId_4841   
95658   TransactionId_91597    BatchId_3503  AccountId_3439   
95659   TransactionId_82501  BatchId_118602  AccountId_4841   
95660  TransactionId_136354   BatchId_70924  AccountId_1346   
95661   TransactionId_35670   BatchId_29317  AccountId_4841   

            SubscriptionId       CustomerId CurrencyCode  CountryCode  \
0       SubscriptionId_887  CustomerId_4406          UGX          256   
1      SubscriptionId_3829  CustomerId_4406          UGX          256   
2       SubscriptionId_2

In [9]:
# Call the standalone function to create aggregated features
aggregates_data= creating_aggregate_features(fe.data)


In [10]:
aggregates_data.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,Total_Transaction_Amount,Average_Transaction_Amount,Transaction_Count,Std_Transaction_Amount
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0,109921.75,923.712185,119,3042.294251
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0,109921.75,923.712185,119,3042.294251
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0,1000.00,500.000000,2,0.000000
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0,228727.20,6019.136842,38,17169.241610
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0,228727.20,6019.136842,38,17169.241610


In [7]:
from Feature_Engineering import  extract_transaction_features
data_with_features = extract_transaction_features(fe.data)


In [8]:
data_with_features.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,TransactionHour,TransactionDay,TransactionMonth,TransactionYear
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0,2,15,11,2018
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,0,2,15,11,2018
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,0,2,15,11,2018
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0,3,15,11,2018
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15 03:34:21+00:00,2,0,3,15,11,2018


##Extracted Transaction Features
The above new columns were added to the DataFrame based on the transaction timestamp (TransactionStartTime):

TransactionHour: The hour of the day when the transaction occurred.

TransactionDay: The day of the month when the transaction occurred.

TransactionMonth: The month when the transaction occurred.

TransactionYear: The year when the transaction occurred

Identify Categorical Columns for Encoding

In [9]:
ef = data_with_features
print(ef.dtypes)

TransactionId                        object
BatchId                              object
AccountId                            object
SubscriptionId                       object
CustomerId                           object
CurrencyCode                         object
CountryCode                           int64
ProviderId                           object
ProductId                            object
ProductCategory                      object
ChannelId                            object
Amount                              float64
Value                                 int64
TransactionStartTime    datetime64[ns, UTC]
PricingStrategy                       int64
FraudResult                           int64
TransactionHour                       int32
TransactionDay                        int32
TransactionMonth                      int32
TransactionYear                       int32
dtype: object


In [10]:
# Identify categorical columns in the 'ef' DataFrame
categorical_columns = ef.select_dtypes(include=['object', 'category']).columns

# Print the list of categorical columns
print("Categorical columns:", categorical_columns)


Categorical columns: Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'ProviderId', 'ProductId', 'ProductCategory',
       'ChannelId'],
      dtype='object')


In [ ]:
from Feature_Engineering import  encoding
ef_encoded = encoding(ef, categorical_columns)


INFO:Feature_Engineering:encoding the categorical variables
INFO:Feature_Engineering:label encoding for ordinal categorical variables
INFO:Feature_Engineering:Checking data types of columns for WOE encoding
INFO:Feature_Engineering:TransactionId                        object
BatchId                              object
AccountId                            object
SubscriptionId                       object
CustomerId                           object
CurrencyCode                         object
CountryCode                           int64
ProviderId                           object
ProductId                            object
ProductCategory                      object
ChannelId                            object
Amount                              float64
Value                                 int64
TransactionStartTime    datetime64[ns, UTC]
PricingStrategy                       int64
FraudResult                           int64
TransactionHour                       int32
TransactionDay      

In [ ]:
eda.identify_missing_values()

In [ ]:
# Identify numerical columns
numerical_columns = ef_encoded.select_dtypes(include=['int64', 'int32', 'float64']).columns
print("Numerical columns:", numerical_columns)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the Min-Max Scaler
min_max_scaler = MinMaxScaler()

# Normalize the numerical features
ef_encoded_normalized = ef_encoded.copy()
ef_encoded_normalized[numerical_columns] = min_max_scaler.fit_transform(ef_encoded[numerical_columns])

print("Data after normalization:\n", ef_encoded_normalized.head())


In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the Standard Scaler
standard_scaler = StandardScaler()

# Standardize the numerical features
ef_encoded_standardized = ef_encoded.copy()
ef_encoded_standardized[numerical_columns] = standard_scaler.fit_transform(ef_encoded[numerical_columns])

print("Data after standardization:\n", ef_encoded_standardized.head())
